<h3>02.CNN으로 MNIST분류하기</h3>

<h4>1. 모델 이해하기</h4>
  
<h5>1. 첫번째 표기 방법</h5>
합성곱(nn.Cov2d) + 활성화 함수(nn.ReLU)를 하나의 합성곱 층으로 보고, 맥스풀링(nn.MaxPoold2d)은 풀링 층으로 별도로 명명합니다.
  
<h5>2. 두번째 표기 방법</h5>  
합성곱(nn.Conv2d) + 활성화 함수(nn.ReLU) + 맥스풀링(nn.MaxPoold2d)을 하나의 합성곱 층으로 봅니다.

다시 말해 풀링도 하나의 층으로 보느냐, 안 보느냐의 문제인데 누가 옳고 틀리냐의 문제는 아니므로, 이번 챕터에서는 편의를 위해 맥스풀링까지도 포함해서 하나의 합성곱 층으로 판단하고 정리해보겠습니다. 다시 말해 두번째 표기 방법을 택하겠습니다.



<h4>2. 모델 구현하기</h4>

In [1]:
#1. 필요한 도구 임포트와 입력의 정의
import torch
import torch.nn as nn


In [2]:
#임의의 텐서 생성
inputs = torch.Tensor(1,1,28,28) #배치크기,채널,높이,너비
print('텐서의 크기: {}'.format(inputs.shape))

텐서의 크기: torch.Size([1, 1, 28, 28])


In [3]:
#2. 합성곱층과 풀링 선언하기
conv1 = nn.Conv2d(1,32,3,padding=1)#3=kernelsize
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [4]:
conv2 = nn.Conv2d(32,64,kernel_size=3,padding=1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [5]:
pool = nn.MaxPool2d(2)
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


In [6]:
#3. 구현체를 연결하여 모델 만들기
out= conv1(inputs)
print(out.shape)

torch.Size([1, 32, 28, 28])


32채널의 28너비 28높이의 텐서가 되었습니다. 32가 나온 이유는 conv1의 out_channel로 32를 지정해주었기 때문입니다. 또한, 28너비 28높이가 된 이유는 패딩을 1폭으로 하고 3 × 3 커널을 사용하면 크기가 보존되기 때문입니다. 이제 이를 맥스풀링을 통과시키고 맥스풀링을 통과한 후의 텐서의 크기를 보겠습니다.



In [7]:
out = pool(out)
print(out.shape)

torch.Size([1, 32, 14, 14])


In [8]:
out = conv2(out)
print(out.shape)

torch.Size([1, 64, 14, 14])


64채널의 14너비 14높이의 텐서가 되었습니다. 64가 나온 이유는 conv2의 out_channel로 64를 지정해주었기 때문입니다. 또한, 14너비 14높이가 된 이유는 패딩을 1폭으로 하고 3 × 3 커널을 사용하면 크기가 보존되기 때문입니다. 이제 이를 맥스풀링을 통과시키고 맥스풀링을 통과한 후의 텐서의 크기를 보겠습니다. 이제 이를 맥스풀링을 통과시키고 맥스풀링을 통과한 후의 텐서의 크기를 보겠습니다.

In [9]:
out = pool(out)
print(out.shape)

torch.Size([1, 64, 7, 7])


In [10]:
#첫 번째 차원인 배치 차원은 그대로 두고 나머지는 펼쳐라
out = out.view(out.size(0),-1)
print(out.shape)

torch.Size([1, 3136])


In [11]:
fc = nn.Linear(3136,10)
out = fc(out)
print(out.shape)

torch.Size([1, 10])


<h4>3.CNN으로 MNIST분류하기</h4>

In [12]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init


In [13]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [14]:
learning_rate = 0.001
training_epochs = 15
batch_size =100


In [15]:
mnist_train = dsets.MNIST(root='MNiST_data/',train=True,transform=transforms.ToTensor(),download=True)
mnist_test = dsets.MNIST(root='MNiST_data/',train=False,transform=transforms.ToTensor(),download=True)

9913344it [00:00, 27083650.37it/s]                             


Extracting MNiST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNiST_data/MNIST/raw



29696it [00:00, 18551392.85it/s]         

Extracting MNiST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNiST_data/MNIST/raw




1649664it [00:00, 7647071.13it/s]                            


Extracting MNiST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNiST_data/MNIST/raw



5120it [00:00, 20088715.14it/s]         

Extracting MNiST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNiST_data/MNIST/raw




/opt/homebrew/Caskroom/miniforge/base/envs/tf/lib/python3.8/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /Users/runner/miniforge3/conda-bld/pytorch-recipe_1643987637853/work/torch/csrc/utils/tensor_numpy.cpp:189.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [16]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,batch_size=batch_size,shuffle=True,drop_last=True)

In [17]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN,self).__init__()

        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1,32,kernel_size=3,stride=1,padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2,stride=2)
        )

        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2,stride=2)
        )

        self.fc = torch.nn.Linear(7*7*64,10,bias=True)

        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out =out.view(out.size(0),-1)
        out = self.fc(out)
        return out

In [18]:
model = CNN().to(device)

In [19]:
criterion = torch.nn.CrossEntropyLoss().to(device)    # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [20]:
total_batch = len(data_loader)
print('총 배치의 수: {}'.format(total_batch))

총 배치의 수: 600


In [22]:
for epoch in range(training_epochs):
    avg_cost=0

    for X,Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis,Y)
        cost.backward()
        optimizer.step()

        avg_cost +=cost/total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))


[Epoch:    1] cost = 0.14996779
[Epoch:    2] cost = 0.0613331422
[Epoch:    3] cost = 0.0462134331
[Epoch:    4] cost = 0.0376325808
[Epoch:    5] cost = 0.0309195984
[Epoch:    6] cost = 0.0255975574
[Epoch:    7] cost = 0.0216998402
[Epoch:    8] cost = 0.0193540957
[Epoch:    9] cost = 0.0155213214
[Epoch:   10] cost = 0.0120942052
[Epoch:   11] cost = 0.0117268357
[Epoch:   12] cost = 0.00946068391


KeyboardInterrupt: 